In [30]:
from aiida import load_profile, orm
load_profile()
from aiida.orm import Dict, StructureData, FolderData
from aiida.plugins import DataFactory, WorkflowFactory
from aiida import orm
from aiida import engine
import pathlib
from ase import io
import nglview
from nglview.adaptor import ASEStructure
import ipywidgets as widgets
import os
import glob
import copy

# Run Parameters:

In [2]:
geom_file = 'sucrose.xyz'
spin = 0
charge = 0
verbose = 4
basis = 'def2svp'
func = 'wb97xd'
orbitals = [180, 181, 182, 183, 184]
cube_files_dir = "cube_files"

# Visualize Geometry:

In [3]:
ase_atoms = io.read(geom_file)
view = nglview.show_ase(ase_atoms)
view

NGLWidget()

# Run Pyscf Plugin:

In [4]:
params = {}
# Structure
params["structure"] = StructureData(ase=ase_atoms)

# Load Code Nodes
params["code"] = orm.load_code(label='pyscf-enroot@pbj-hc44-enroot')

# Autocas Parameters:
params["parameters"] = Dict({
    'structure': {
        'basis': basis,
    },
    'mean_field': {
        'xc': func,
        'method': 'RKS',
    },
    'cubegen': {
        'orbitals': {
            'indices': orbitals,
        },
        'density': {},
        'mep': {}
    },
})

PyscfWorkChain = WorkflowFactory('pyscf.base')

result, process_node = engine.run_get_node(PyscfWorkChain,**{ 'pyscf':params })
print(result["parameters"].get_dict())
result

06/13/2023 02:48:40 PM <2614448> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [55843|PyscfBaseWorkChain|run_process]: launching PyscfCalculation<55844> iteration #1
06/13/2023 02:48:40 PM <2614448> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [55843|PyscfBaseWorkChain|results]: work chain completed after 1 iterations
06/13/2023 02:48:40 PM <2614448> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [55843|PyscfBaseWorkChain|on_terminated]: remote folders will not be cleaned


{'timings': {'total': 117.274159039, 'cubegen': {'mep': 117.274123236, 'density': 0.90311986799998, 'orbitals': 3.175127645}, 'mean_field': 73.079781012}, 'mean_field': {'forces': [[-1.2599440437227, -1.1966534871137, 0.81223384159965], [0.86985341342432, 0.48845740161561, -1.2162519402954], [0.78289941048396, -0.53999528258018, 0.2390518007903], [-0.01460433945375, 0.079759019267493, -0.061077236679653], [0.015128219617558, 0.1703366831574, 0.18637247710994], [0.15379886969931, -0.39711602213324, -0.3182013093163], [0.01112693453039, 0.24784490726282, -0.12587710792071], [0.21488518600329, -0.10724320441126, -0.39623150224585], [-0.23735096852809, -0.47917416053916, -0.087046258965144], [-0.036004340576589, 0.091762071537472, 0.058982873571656], [0.038301989000423, 0.14680849279331, -0.079495668634408], [0.58787171972335, 0.64539609745249, -0.39757389486802], [-0.58008372384798, -0.36238231677616, 1.722810291023], [-0.97203877653483, 0.55073700270214, -0.92480570440584], [0.8835197363

{'remote_folder': <RemoteData: uuid: c0976502-a45e-47c7-902a-96708747db0d (pk: 55845)>,
 'retrieved': <FolderData: uuid: b80bc2c9-685d-436a-9ef7-1637eb739eaa (pk: 55846)>,
 'parameters': <Dict: uuid: ad19e2b6-c1c6-45f0-81bd-a493d4495811 (pk: 55854)>,
 'cubegen': AttributeDict({'orbitals': AttributeDict({'mo_180': <SinglefileData: uuid: e70332cd-b6f9-4d8c-84c5-72c5f145cc3f (pk: 55847)>, 'mo_181': <SinglefileData: uuid: dd98c665-ff0d-4432-acd3-023a8d384e27 (pk: 55848)>, 'mo_182': <SinglefileData: uuid: effd6e48-703c-44d7-866e-7a3a281ccb3d (pk: 55849)>, 'mo_183': <SinglefileData: uuid: 988ff36d-b166-4085-98c1-4f9bc5af8641 (pk: 55850)>, 'mo_184': <SinglefileData: uuid: 6bed396b-6aae-4eae-b32e-3f5a86766bfd (pk: 55851)>}), 'density': <SinglefileData: uuid: 1303a991-5ce3-452c-ad4a-d3d856b8ee9b (pk: 55852)>, 'mep': <SinglefileData: uuid: b82c0641-b79b-48ed-bc0f-8ece6357eae2 (pk: 55853)>})}

In [6]:
print("Top Level Keys: ", result["cubegen"].keys())
print("MO Cube Keys: ", result["cubegen"]["orbitals"].keys())

Top Level Keys:  dict_keys(['orbitals', 'density', 'mep'])
MO Cube Keys:  dict_keys(['mo_180', 'mo_181', 'mo_182', 'mo_183', 'mo_184'])


In [35]:
mo_dir = cube_files_dir + "/mos"
if not os.path.exists(cube_files_dir):
    os.mkdir(cube_files_dir)

if not os.path.exists(mo_dir):
    os.mkdir(mo_dir)

density_cube_file = cube_files_dir + "/density.cube"
with open(density_cube_file,'w') as f:
    f.write(result["cubegen"]["density"].get_content())

mep_cube_file = cube_files_dir + "/mep.cube"
with open(mep_cube_file,'w') as f:
    f.write(result["cubegen"]["density"].get_content())

for mo in result["cubegen"]["orbitals"].keys():
    with open(mo_dir+f"/{mo}.cube",'w') as f:
        f.write(result["cubegen"]["orbitals"][mo].get_content())

# Visualize MOs:

In [93]:
mo_cube_files = glob.glob(mo_dir+"/*.cube")
mo_dropdown = widgets.Dropdown(
    options=mo_cube_files,
    value=mo_cube_files[0],
    description='MO Cube Files',
    disabled=False,
)

iso_level = 1.2
negative_color = "#ff5c39"
positive_color = "#0078d4"

mo_view = nglview.show_ase(ase_atoms)
pos_comp = mo_view.add_component(mo_dropdown.value, ext="cube")
neg_comp = mo_view.add_component(mo_dropdown.value, ext="cube")
pos_comp.update_surface(isolevel=iso_level, color=positive_color)
neg_comp.update_surface(isolevel=-iso_level,  color=negative_color)
display(mo_dropdown,mo_view)


def mo_on_change( value=None ):
    global mo_view
    uuids = copy.deepcopy(mo_view._ngl_component_ids)
    for uuid in uuids:
        mo_view.remove_component(uuid)
    struc = ASEStructure(ase_atoms)
    mo_view.add_component(struc)
    pos_comp = mo_view.add_component(mo_dropdown.value, ext="cube")
    neg_comp = mo_view.add_component(mo_dropdown.value, ext="cube")
    pos_comp.update_surface(isolevel=iso_level, color=positive_color)
    neg_comp.update_surface(isolevel=-iso_level,  color=negative_color)

mo_dropdown.observe(mo_on_change,names="value")

Dropdown(description='MO Cube Files', options=('cube_files/mos/mo_180.cube', 'cube_files/mos/mo_181.cube', 'cu…

NGLWidget()

# Visualize Density:

In [85]:
den_slider = widgets.FloatSlider(
    value=0.1,
    min=0.01,
    max=0.5,
    step=0.01,
    description='Isovalue:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

positive_color = "#0078d4"

den_view = nglview.show_ase(ase_atoms)
den_pos_comp = den_view.add_component(density_cube_file, ext="cube")
den_pos_comp.update_surface(isolevel=den_slider.value, color=positive_color, opacity=0.8)
display(den_slider,den_view)


def den_on_change( value=None ):
    global den_view
    uuids = copy.deepcopy(den_view._ngl_component_ids)
    for uuid in uuids:
        den_view.remove_component(uuid)
    struc = ASEStructure(ase_atoms)
    den_view.add_component(struc)
    pos_comp = den_view.add_component(density_cube_file, ext="cube")
    pos_comp.update_surface(isolevel=value['new'], color=positive_color, opacity=0.8)

den_slider.observe(den_on_change,names="value")

FloatSlider(value=0.1, continuous_update=False, description='Isovalue:', max=0.5, min=0.01, step=0.01)

NGLWidget()